In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import os
import zipfile
from datetime import datetime
import time
import copy

classes = ['Pastos', 'planta_daninha', 'planta_toxicas']

class EmbeddingDataset(Dataset):
    def __init__(self, csv_file, num_classes=3):
        self.data = pd.read_csv(csv_file, header=0)
        self.num_classes = num_classes
        self.class_to_idx = {classes[i]: i for i in range(num_classes)}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        label_str, embedding_str = self.data.iloc[idx]
        label = self.class_to_idx[label_str]
        embedding = np.fromstring(embedding_str.strip('[]'), sep=' ')
        embedding = torch.tensor(embedding, dtype=torch.float32)
        return embedding, label

class EmbeddingClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmbeddingClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

def plot_fold_history(histories, output_dir):
    """
    Plota o histórico de treinamento para todos os folds
    """
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    for fold, history in enumerate(histories):
        plt.plot(history['train_acc'], label=f'Treino Fold {fold+1}')
        plt.plot(history['val_acc'], label=f'Val Fold {fold+1}', linestyle='--')
    plt.title('Acurácia do Modelo por Fold')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    for fold, history in enumerate(histories):
        plt.plot(history['train_loss'], label=f'Treino Fold {fold+1}')
        plt.plot(history['val_loss'], label=f'Val Fold {fold+1}', linestyle='--')
    plt.title('Perda do Modelo por Fold')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_history.png'))
    plt.close()

def plot_fold_metrics(fold_scores, output_dir):
    """
    Plota as métricas finais de cada fold em um gráfico de barras
    """
    plt.figure(figsize=(10, 6))
    folds = range(1, len(fold_scores) + 1)
    plt.bar(folds, fold_scores)
    plt.axhline(y=np.mean(fold_scores), color='r', linestyle='--', label='Média')
    
    plt.title('Acurácia por Fold')
    plt.xlabel('Número do Fold')
    plt.ylabel('Acurácia')
    plt.ylim([0, 1])
    
    for i, v in enumerate(fold_scores):
        plt.text(i + 1, v + 0.01, f'{v:.4f}', ha='center')
    
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'fold_metrics.png'))
    plt.close()

def plot_confusion_matrix(y_true, y_pred, output_dir):
    """
    Plota e salva a matriz de confusão
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()

def save_results(fold_scores, classification_rep, fold_times, total_time, output_dir):
    """
    Salva os resultados em arquivos de texto
    """
    with open(os.path.join(output_dir, 'fold_accuracies.txt'), 'w') as f:
        f.write("Resultados por Fold:\n")
        for i, (score, time_taken) in enumerate(zip(fold_scores, fold_times)):
            f.write(f"Fold {i+1}:\n")
            f.write(f"  Acurácia: {score:.4f}\n")
            f.write(f"  Tempo de treinamento: {time_taken:.2f} segundos ({time_taken/60:.2f} minutos)\n")
        
        f.write(f"\nMédia de Acurácia: {np.mean(fold_scores):.4f}")
        f.write(f"\nDesvio Padrão de Acurácia: {np.std(fold_scores):.4f}")
        f.write(f"\n\nTempo médio por fold: {np.mean(fold_times):.2f} segundos ({np.mean(fold_times)/60:.2f} minutos)")
        f.write(f"\nTempo total de treinamento: {total_time:.2f} segundos ({total_time/60:.2f} minutos)")

    with open(os.path.join(output_dir, 'classification_report.txt'), 'w') as f:
        f.write(classification_rep)

def create_zip_archive(output_dir, zip_filename):
    """
    Cria um arquivo zip com todos os resultados
    """
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, arcname)

def train_evaluate_fold(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    """
    Treina e avalia o modelo para um fold específico
    """
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Record history
        history['train_loss'].append(train_loss / len(train_loader))
        history['train_acc'].append(train_correct / train_total)
        history['val_loss'].append(val_loss / len(val_loader))
        history['val_acc'].append(val_correct / val_total)
        
    return history, val_correct / val_total

def main():
    # Configurações
    csv_file = '../../embeddings_with_augmentation.csv'
    k_folds = 10
    num_epochs = 100
    batch_size = 32
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Criar diretório para resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f'resultados_kfold_{timestamp}'
    os.makedirs(output_dir, exist_ok=True)
    
    # Carregar dataset
    print("Carregando dados...")
    dataset = EmbeddingDataset(csv_file)
    input_dim = dataset[0][0].shape[0]
    
    # Inicializar K-Fold
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    
    # Métricas e históricos
    fold_histories = []
    fold_scores = []
    fold_times = []
    all_predictions = []
    all_true_labels = []
    
    # Marca o início do treinamento total
    total_start_time = time.time()
    
    # Loop através dos folds
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\nFold {fold + 1}/{k_folds}")
        print("-" * 20)
        
        # Preparar dataloaders
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
        
        # Inicializar modelo, critério e otimizador
        model = EmbeddingClassifier(input_dim, len(classes)).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Marca o início do treinamento do fold
        fold_start_time = time.time()
        
        # Treinar e avaliar o fold
        history, fold_accuracy = train_evaluate_fold(
            model, train_loader, val_loader, criterion, optimizer, device, num_epochs
        )
        
        # Calcular o tempo do fold
        fold_time = time.time() - fold_start_time
        fold_times.append(fold_time)
        
        # Coletar predições para a matriz de confusão
        model.eval()
        fold_preds = []
        fold_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                fold_preds.extend(predicted.cpu().numpy())
                fold_labels.extend(labels.numpy())
        
        all_predictions.extend(fold_preds)
        all_true_labels.extend(fold_labels)
        
        # Armazenar resultados
        fold_histories.append(history)
        fold_scores.append(fold_accuracy)
        
        print(f"Fold {fold + 1} - Acurácia: {fold_accuracy:.4f}")
        print(f"Tempo de treinamento do fold: {fold_time:.2f} segundos ({fold_time/60:.2f} minutos)")
    
    # Calcular o tempo total de treinamento
    total_time = time.time() - total_start_time
    
    # Gerar relatório de classificação
    report = classification_report(all_true_labels, all_predictions, 
                                 target_names=classes, digits=4)
    
    # Gerar visualizações e salvar resultados
    print("\nGerando relatórios e visualizações...")
    plot_fold_history(fold_histories, output_dir)
    plot_fold_metrics(fold_scores, output_dir)
    plot_confusion_matrix(all_true_labels, all_predictions, output_dir)
    save_results(fold_scores, report, fold_times, total_time, output_dir)
    
    # Criar arquivo zip
    zip_filename = f'resultados_kfold_{timestamp}.zip'
    create_zip_archive(output_dir, zip_filename)
    
    print(f"\nResultados da Validação Cruzada:")
    print(f"Acurácia média: {np.mean(fold_scores):.4f} (+/- {np.std(fold_scores):.4f})")
    print(f"Tempo médio por fold: {np.mean(fold_times):.2f} segundos ({np.mean(fold_times)/60:.2f} minutos)")
    print(f"Tempo total de treinamento: {total_time:.2f} segundos ({total_time/60:.2f} minutos)")
    print(f"\nTodos os resultados foram salvos em '{zip_filename}'")

if __name__ == "__main__":
    main()